## 1. Message Body Type Registration

The default message body type is stored in `src/gai/messages/typing.py`. To extend the message body types, you can register using `@register_body` decorator and update the message class using `get_message_cls()`.

This tests are to confirm that new message body types can be registered dynamically and that the system recognizes them as valid message body types.

### a) Serializing and De-serializing a Built-in Message Body Type

This is to confirm that body type successfully validate DefaultBodyPydantic as a built-in message body type and is able to serialize and deserialize it correctly.



In [1]:
from gai.messages.typing import MessagePydantic, DefaultBodyPydantic

# This is a test to confirm that the DefaultBodyPydantic can be used as a message body type.

message = MessagePydantic(
    body = DefaultBodyPydantic(
        content="hello"
    )
)
print(message)

# This is to confirm that the DefaultBodyPydantic can be serialized and deserialized correctly.

state_json = {
    'id': 'a0e5f98c-f6eb-47de-a6e2-387510d970f9', 
    'header': {
        'sender': 'User', 
        'recipient': 'Assistant', 
        'timestamp': 1752213128.0972457, 
        'order': 0
    }, 'body': {
        'type': 'default',
        'content': 'Tell me a one paragraph story'
    }
}
message = MessagePydantic(**state_json)
print(message)
assert isinstance(message.body,DefaultBodyPydantic)


id='aba4ae55-ee9a-4481-aad1-e806692afbbd' header=MessageHeaderPydantic(sender='User', recipient='Assistant', timestamp=1752490758.8517375, order=0) body=DefaultBodyPydantic(type='default', content='hello')
id='a0e5f98c-f6eb-47de-a6e2-387510d970f9' header=MessageHeaderPydantic(sender='User', recipient='Assistant', timestamp=1752213128.0972457, order=0) body=DefaultBodyPydantic(type='default', content='Tell me a one paragraph story')


### b) Serializing and De-serializing a Custom Message Body Type

This is to confirm that body type recognizes the custom body type `PingBodyPydantic` dynamically as a message body type and is able to serialize and deserialize it correctly.




In [2]:
from gai.messages.typing import get_message_cls,register_body
from pydantic import BaseModel
from typing import Literal

# This is to register a custom message body dynamically

# This is to tag the body type as a message body type
@register_body
class PingBodyPydantic(BaseModel):
    type: Literal["Ping"] = "Ping"
    content: str

# This is to update the message class to include the new body type
MessagePydantic = get_message_cls()

# This is to use to updated message class with the new body type
message = MessagePydantic(
    body = PingBodyPydantic(content="Ping")
)
print(message)

jsoned = {
    'id': 'a0e5f98c-f6eb-47de-a6e2-387510d970f9',
    'header': {
        'sender': 'User',
        'recipient': 'Assistant',
        'timestamp': 1752213128.0972457,
        'order': 0
    },
    'body': {
        'type': 'Ping',
        'content': 'Ping'
    }
 }
message = MessagePydantic(**jsoned)
print(message)
assert isinstance(message.body,PingBodyPydantic)

id='418e46fb-7a51-4108-9beb-13b854930db3' header=MessageHeaderPydantic(sender='User', recipient='Assistant', timestamp=1752490761.72441, order=0) body=PingBodyPydantic(type='Ping', content='Ping')
id='a0e5f98c-f6eb-47de-a6e2-387510d970f9' header=MessageHeaderPydantic(sender='User', recipient='Assistant', timestamp=1752213128.0972457, order=0) body=PingBodyPydantic(type='Ping', content='Ping')


---

## 2. Message Helper

This tests the "create_message" and "convert_to_chat_messages" functions in the `gai.messages.helper` module.

### a) "create_message"

Create a default body type message using "create_message" function.

In [3]:
from gai.messages import message_helper

# Create a default type message using the message helper
default_type_message = message_helper.create_message(role="user", content="What is the capital of France?")

assert default_type_message.body.type == "default"
print(default_type_message)

id='afacd1e5-d4a9-4818-8308-68a2995bfe0e' header=MessageHeaderPydantic(sender='User', recipient='Assistant', timestamp=1752490764.292277, order=0) body=DefaultBodyPydantic(type='default', content='What is the capital of France?')


### b) "json" / "unjson"

Serialize and deserialize a list of messages of different body types using `json` and `unjson` functions.

In [4]:
from pydantic import BaseModel
from typing import Literal
from gai.messages import message_helper
from gai.messages.typing import register_body, get_message_cls

@register_body
class SendBodyPydantic(BaseModel):
    type: Literal["send"] = "send"
    recipient: str
    content: str
    
@register_body
class ReplyBodyPydantic(BaseModel):
    type: Literal["reply"] = "reply"
    sender: str
    recipient: str
    chunk_no: int
    chunk: str

MessagePydantic = get_message_cls()

gai_messages = [
    MessagePydantic(
        header={"sender": "User", "recipient": "Sara", "timestamp": 1752213128.0972457, "order": 0},
        body=SendBodyPydantic(recipient="Sara", content="Tell me a story about a brave knight.")
    ),
    MessagePydantic(
        header={"sender": "Sara", "recipient": "User", "timestamp": 1752213128.0972457, "order": 1},
        body=ReplyBodyPydantic(sender="Sara", recipient="User", chunk_no=0, chunk="<eom>", content="Once upon a time, a brave knight saved a kingdom from a dragon.")
    )
]

### Serialize list of messages to JSON
jsoned = message_helper.json(gai_messages)
print(jsoned)

### Deserialize JSON back to list of messages using the dynamic MessagePydantic class
deserialized_messages = message_helper.unjson(jsoned, MessagePydantic)
for msg in deserialized_messages:
    print(msg)
    assert isinstance(msg, MessagePydantic)

[
    {
        "id": "c7456134-a25f-4330-814e-22ab19d6a001",
        "header": {
            "sender": "User",
            "recipient": "Sara",
            "timestamp": 1752213128.0972457,
            "order": 0
        },
        "body": {
            "type": "send",
            "recipient": "Sara",
            "content": "Tell me a story about a brave knight."
        }
    },
    {
        "id": "15a33a10-d744-4d2c-991b-30341be2e72d",
        "header": {
            "sender": "Sara",
            "recipient": "User",
            "timestamp": 1752213128.0972457,
            "order": 1
        },
        "body": {
            "type": "reply",
            "sender": "Sara",
            "recipient": "User",
            "chunk_no": 0,
            "chunk": "<eom>"
        }
    }
]
id='c7456134-a25f-4330-814e-22ab19d6a001' header=MessageHeaderPydantic(sender='User', recipient='Sara', timestamp=1752213128.0972457, order=0) body=SendBodyPydantic(type='send', recipient='Sara', content='Tell m

---

## 3. Message Storage

a) Create dynamic message class

In [1]:
# For Troubleshooting Only: Print the body type of each message

from gai.messages.typing import _BODY_CLASSES
for cls in _BODY_CLASSES:
    print(cls)

<class 'gai.messages.typing.DefaultBodyPydantic'>
<class 'gai.messages.typing.MonologueBodyPydantic'>


In [2]:
from pydantic import BaseModel
from typing import Literal, Optional
from gai.messages.typing import register_body, get_message_cls

@register_body
class SendBodyPydantic(BaseModel):
    type: Literal["send"] = "send"
    role: str
    content: str
    
@register_body
class ReplyBodyPydantic(BaseModel):
    type: Literal["reply"] = "reply"
    role: str
    chunk_no: int
    chunk: str
    content: Optional[str] = None  # Optional content field for reply messages

MessagePydantic = get_message_cls()


b) Reset temp dir and create a new message store

In [3]:
import os
from gai.messages.message_store import MessageStore
from gai.lib.tests import make_local_tmp
here = make_local_tmp()
file_path = os.path.join(here,"messages.json")
store = MessageStore(file_path=file_path, MessagePydantic_cls=MessagePydantic)

c) Insert messages

In [4]:
store.reset()

# Insert a single user message

message = MessagePydantic(**{
    'id': 'a0e5f98c-f6eb-47de-a6e2-387510d970f9', 
    'header': {
        'sender': 'User', 
        'recipient': 'Assistant', 
    }, 'body': {
        'type': 'send', 
        'role': 'user', 
        'content': 'Tell me a one paragraph story'
    }
})
store.insert_message(message)

# Insert a stream of assistant messages

messages = [
    MessagePydantic(**{
        'id': 'b1e5f98c-f6eb-47de-a6e2-387510d970f9', 
        'header': {
            'sender': 'Assistant',
            'recipient': 'User',
        }, 'body': {
            'type': 'reply',
            'role': 'assistant',
            'chunk_no': 0,
            'chunk': 'Once upon a time,'
        }
    }),
    MessagePydantic(**{
        'id': 'abbc7961-45dc-4973-aaf4-a6224ed35d37', 
        'header': {
            'sender': 'Assistant',
            'recipient': 'User',
        }, 'body': {
            'type': 'reply',
            'role': 'assistant',
            'chunk_no': 1,
            'chunk': ' a brave knight saved a kingdom from a dragon.'
        }
    }),
    MessagePydantic(**{
        'id': 'ae3d1495-3aac-4aa8-a833-b753720c9851', 
        'header': {
            'sender': 'Assistant',
            'recipient': 'User',
        }, 'body': {
            'type': 'reply',
            'role': 'assistant',
            'chunk_no': 2,
            'chunk': '<eom>',
            'content': 'Once upon a time, a brave knight saved a kingdom from a dragon.'
        }
    })
]
store.bulk_insert_messages(messages)

# Should show 4 messages

messages = store.list_messages()
for message in messages:
    print(message)
    assert isinstance(message, MessagePydantic)    
assert len(messages) == 4, f"Expected 4 messages, got {len(messages)}"


id='a0e5f98c-f6eb-47de-a6e2-387510d970f9' header=MessageHeaderPydantic(sender='User', recipient='Assistant', timestamp=1752386688.1068888, order=0) body=SendBodyPydantic(type='send', role='user', content='Tell me a one paragraph story')
id='b1e5f98c-f6eb-47de-a6e2-387510d970f9' header=MessageHeaderPydantic(sender='Assistant', recipient='User', timestamp=1752386688.1091309, order=1) body=ReplyBodyPydantic(type='reply', role='assistant', chunk_no=0, chunk='Once upon a time,', content=None)
id='abbc7961-45dc-4973-aaf4-a6224ed35d37' header=MessageHeaderPydantic(sender='Assistant', recipient='User', timestamp=1752386688.1091433, order=2) body=ReplyBodyPydantic(type='reply', role='assistant', chunk_no=1, chunk=' a brave knight saved a kingdom from a dragon.', content=None)
id='ae3d1495-3aac-4aa8-a833-b753720c9851' header=MessageHeaderPydantic(sender='Assistant', recipient='User', timestamp=1752386688.1091464, order=3) body=ReplyBodyPydantic(type='reply', role='assistant', chunk_no=2, chunk='

get_message

In [5]:
store.get_message(message_id="a0e5f98c-f6eb-47de-a6e2-387510d970f9")

MessagePydantic(id='a0e5f98c-f6eb-47de-a6e2-387510d970f9', header=MessageHeaderPydantic(sender='User', recipient='Assistant', timestamp=1752386688.1068888, order=0), body=SendBodyPydantic(type='send', role='user', content='Tell me a one paragraph story'))

update_message

In [6]:
message_json = {
    'id': 'a0e5f98c-f6eb-47de-a6e2-387510d970f9', 
    'header': {
        'sender': 'User', 
        'recipient': 'Agent1', 
    }, 'body': {
        'type': 'send', 
        'role': 'user', 
        'content': 'Tell me a one paragraph story'
    }
}
store.update_message(message=MessagePydantic(**message_json))
message = store.get_message(message_id="a0e5f98c-f6eb-47de-a6e2-387510d970f9")
assert message.header.recipient == "Agent1", "Recipient should be updated to Agent1"

delete_message

In [7]:
messages = store.list_messages()
assert len(messages) == 4, f"There should be one message in the store but got {len(messages)}"

store.delete_message(id="a0e5f98c-f6eb-47de-a6e2-387510d970f9")
messages = store.list_messages()
assert len(messages) == 3, "There should be no messages in the store after deletion"

extract_recap

In [8]:
messages = [
    MessagePydantic(**{
        'id': 'a0e5f98c-f6eb-47de-a6e2-387510d970f9', 
        'header': {
            'sender': 'User', 
            'recipient': 'Assistant', 
        }, 'body': {
            'type': 'send', 
            'role': 'user', 
            'content': 'Tell me a one paragraph story'
        }
    }),    
    MessagePydantic(**{
        'id': 'b1e5f98c-f6eb-47de-a6e2-387510d970f9', 
        'header': {
            'sender': 'Assistant',
            'recipient': 'User',
        }, 'body': {
            'type': 'reply',
            'role': 'assistant',
            'chunk_no': 0,
            'chunk': 'Once upon a time,'
        }
    }),
    MessagePydantic(**{
        'id': 'abbc7961-45dc-4973-aaf4-a6224ed35d37', 
        'header': {
            'sender': 'Assistant',
            'recipient': 'User',
        }, 'body': {
            'type': 'reply',
            'role': 'assistant',
            'chunk_no': 1,
            'chunk': ' a brave knight saved a kingdom from a dragon.'
        }
    }),
    MessagePydantic(**{
        'id': 'ae3d1495-3aac-4aa8-a833-b753720c9851', 
        'header': {
            'sender': 'Assistant',
            'recipient': 'User',
        }, 'body': {
            'type': 'reply',
            'role': 'assistant',
            'chunk_no': 2,
            'chunk': '<eom>',
            'content': 'Once upon a time, a brave knight saved a kingdom from a dragon.'
        }
    })
]

from gai.messages import message_helper
message_helper.extract_recap(messages=messages,last_n=0,max_recap_size=1000)


'User: Tell me a one paragraph story\nAssistant: Once upon a time, a brave knight saved a kingdom from a dragon.'